## Problems
## TODO:
### understand the .rds import
### find out where those ~600 samples from the paper went... 
##### -> when were from other papers with infectious data. We could use that as well but i guess the focus on corona is higher.. 
### ensembl IDs

## attached .rds file: 86 subjects only! GEO accession for more info GSE152641 
Total RNA sequencing from whole blood of 62 COVID-19 patients and 24 healthy controls was compared to publicly available expression data from other infections."

- Whole blood 
62 COVID
24 HD

## inflammatrix.rds vs GSE152641_Inflammatix_COVID19_counts_entrez.csv.gz	

In [110]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.io import mmread


# inflammatrix.rds

In [111]:
import pyreadr
# Thanks to: https://stackoverflow.com/questions/40996175/loading-a-rds-file-in-pandas 

thair_data = pyreadr.read_r('/home/eliasschreiner/PROJEKT/2021_Thair_COVID/covid.inflammatix86.exprlist.rds') # also works for RData

# done! 
# result is a dictionary where keys are the name of objects and the values python
# objects. In the case of Rds there is only one object with None as key
#df = thair_data[None] # extract the pandas data frame 
thair_data

OrderedDict()

In [112]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

pandas2ri.activate()

readRDS = robjects.r['readRDS']
r_data = readRDS('/home/eliasschreiner/PROJEKT/2021_Thair_COVID/covid.inflammatix86.exprlist.rds')
readRDS


<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x7f290147c0d0> [3]
R classes: ('function',)

In [113]:
print(r_data[0])
print(r_data[1])
print(r_data[2])


if isinstance(r_data, robjects.vectors.ListVector):
    df_list = []
    for item in r_data:
        if isinstance(item, robjects.vectors.DataFrame):
            df_list.append(pandas2ri.rpy2py(item))
        else:
            # Handle other types if necessary
            pass
    # Example: Combine all dataframes in the list into a single dataframe (if they are compatible)
    df = pd.concat(df_list, ignore_index=True)
else:
    # Assuming r_data is a DataFrame-like object
    df = pandas2ri.rpy2py(r_data)
df
type(r_data)
#df = pandas2ri.ri2py(r_data)
# do something with the dataframe

[[0.634612   0.61739435 0.69599908 ... 0.59235934 0.56837607 0.54091514]
 [0.52559669 0.52559669 0.52559669 ... 0.61666198 0.52559669 0.52559669]
 [1.06828365 1.10848323 1.07198903 ... 1.11930872 1.14026099 1.13653577]
 ...
 [0.70291304 0.72581236 0.76439531 ... 0.68339396 0.66046261 0.61321769]
 [0.53938502 0.56447889 0.53938502 ... 0.53938502 0.53938502 0.53938502]
 [0.69771149 0.68545223 0.73441705 ... 0.71402081 0.66632246 0.77896879]]
             sampleid condition A_a Patient_accession_                LOS
BRH1562565 BRH1562565      CTRL  NA                                      
BRH1562570 BRH1562570      CTRL  NA                                      
BRH1562573 BRH1562573      CTRL  NA                                      
BRH1562575 BRH1562575      CTRL  NA                                      
BRH1562576 BRH1562576      CTRL  NA                                      
BRH1562577 BRH1562577      CTRL  NA                                      
BRH1562578 BRH1562578      CTRL  NA   

rpy2.robjects.vectors.ListVector

In [114]:
def r_df_to_pd_df(r_df, index=0):
    print(type(r_df[index]))
    from rpy2.robjects import pandas2ri
    pandas2ri.activate()
    return  pandas2ri.rpy2py(r_df[index])
    
    
metadata_df = r_df_to_pd_df(r_data, index=1) #metadata
print(metadata_df.shape)
metadata_df.head()

<class 'rpy2.robjects.vectors.DataFrame'>
(86, 63)


,sampleid,condition,A_a,Patient_accession_,LOS,Days_between_onset_symptoms_and_sampling,Days_between_intubation_and_sampling,Days_between_hospital_admission_and_intubation,Age__years_,Gender,...,libtype,percent_mapped,num_processed,num_mapped,pct.mt,pct.globin,cohort,subjectid,Days_post_symptom_onset,sex
BRH1562565,BRH1562565,CTRL,-2147483648,,,-2147483648,,-2147483648,-2147483648,,...,ISR,30.35,48706235,14782578,0.0185,0.00136,adult,BRH1562565,-2147483648,M
BRH1562570,BRH1562570,CTRL,-2147483648,,,-2147483648,,-2147483648,-2147483648,,...,ISR,32.74,61099020,20004349,0.0191,0.00311,adult,BRH1562570,-2147483648,M
BRH1562573,BRH1562573,CTRL,-2147483648,,,-2147483648,,-2147483648,-2147483648,,...,ISR,34.67,50801386,17611206,0.0182,0.00215,adult,BRH1562573,-2147483648,M
BRH1562575,BRH1562575,CTRL,-2147483648,,,-2147483648,,-2147483648,-2147483648,,...,ISR,36.25,51143071,18539981,0.0138,0.03070,adult,BRH1562575,-2147483648,M
BRH1562576,BRH1562576,CTRL,-2147483648,,,-2147483648,,-2147483648,-2147483648,,...,ISR,32.29,44853997,14482473,0.0211,0.00122,adult,BRH1562576,-2147483648,M


### preproc steps (by papers): 

Normalization and voom transformation of RNAseq counts

Low-expressed genes were filtered using the following cutoff: max counts per million
(CPM) less than 5 across all 86 samples. Normalization factors were obtained using
edgeR’s Trimmed Mean of M values (TMM) method (R package v.3.28.0) (Robinson,
McCarthy and Smyth, 2009). The voom method (limma R package v.3.41.18) was then
used to transform counts into normalized log2-CPM (Supplementary Figure 1)(Ritchie
et al., 2015). 

Non-COVID-19 viral dataset processing

Raw microarray data for each dataset was renormalized (when available) using
standardized methods. Affymetrix arrays were renormalized using the robust multichip
average (RMA) method. Illumina, Agilent, GE, and other commercial arrays were
renormalized via normal-exponential background correction followed by quantile
normalization. Data were log2-transformed. Probe to gene (Entrez ID) summarization
was performed within each study using the mean signal intensity for probes mapping
to a single gene. While there is no consensus in the community, we have used this
method across a multitude of studies, being that if more than one probe mapped to a
specific gene, probes were summarized with a fixed-effects model because a gene
within a sample can have only one expression value(Ramasamy et al., 2008).

In [115]:
preproc_array = list(r_data[0])
preproc_df = pd.DataFrame(preproc_array)
print(preproc_df.shape)
preproc_df.head()

(48246, 86)


,0,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,85
0,0.634612,0.617394,0.695999,0.702577,0.644839,0.675929,0.608621,0.685314,0.595570,0.599146,...,0.574350,0.736129,0.537230,0.566870,0.655854,0.559343,0.594192,0.592359,0.568376,0.540915
1,0.525597,0.525597,0.525597,0.525597,0.525597,0.525597,0.525597,0.525597,0.602239,0.525597,...,0.525597,0.525597,0.592644,0.525597,0.636609,0.525597,0.525597,0.616662,0.525597,0.525597
2,1.068284,1.108483,1.071989,1.081105,1.068706,1.106738,1.099966,1.073934,1.089763,1.046110,...,1.106220,1.203061,1.089094,1.154426,1.132354,1.095921,1.090468,1.119309,1.140261,1.136536
3,1.081746,1.118465,1.117197,1.118340,1.132837,1.107274,1.109919,1.064358,1.092392,1.106659,...,1.148825,1.119218,1.073576,1.180907,1.084621,1.131179,1.089568,1.102321,1.086364,1.116728
4,0.960682,0.924492,0.893195,0.998440,0.876122,0.996598,0.908274,0.913989,0.889657,0.901616,...,0.968062,0.893496,1.032565,0.886781,0.895590,1.016370,0.831301,0.842301,0.855419,0.890305


In [116]:
counts_array = list(r_data[2])
counts_df = pd.DataFrame(counts_array)
print(counts_df.shape)
counts_df.head()

(59034, 86)


,0,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,85
0,10.0,10.0,22.0,22.0,11.0,16.0,8.0,18.0,6.0,7.0,...,2.0,24.0,1.0,2.0,10.0,2.0,4.0,4.0,2.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,1.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0
2,175.0,277.0,194.0,191.0,169.0,210.0,242.0,179.0,213.0,184.0,...,116.0,290.0,139.0,176.0,189.0,154.0,146.0,176.0,158.0,159.0
3,460.0,725.0,614.0,579.0,596.0,520.0,631.0,412.0,530.0,632.0,...,366.0,465.0,311.0,517.0,358.0,464.0,356.0,396.0,290.0,355.0
4,163.0,166.0,116.0,208.0,92.0,197.0,137.0,121.0,113.0,134.0,...,92.0,87.0,177.0,65.0,83.0,172.0,52.0,58.0,51.0,66.0


# GSE152641_Inflammatix_COVID19_counts_entrez.csv.gz	
### This seems to be a bit better. The amount of entries matches rather the ensamble IDs 
### Also, i think for the -rds files we got some kind of high quality reads, i did nopt really find a good emplaination for it tho ... 

In [117]:
path = '/var/datasets/single_papers/2021_thair_covid/GSE152641_Inflammatix_COVID19_counts_entrez.csv'
df_csv = pd.read_csv(path)
print(df_csv)
df_csv.head()

       Unnamed: 0  IMX_sample00001  IMX_sample00002  IMX_sample00003  \
0               1               12                1               10   
1               2             2670             2432              781   
2               9               77              129               87   
3              10                0                0                0   
4              12                0                4                0   
...           ...              ...              ...              ...   
20455   114108587                5                9                8   
20456   114483833                0                1                0   
20457   114483834                0                0                0   
20458   114841035                0                0                0   
20459   114841037                0                0                0   

       IMX_sample00004  IMX_sample00005  IMX_sample00006  IMX_sample00007  \
0                   12                4               12  

,Unnamed: 0,IMX_sample00001,IMX_sample00002,IMX_sample00003,IMX_sample00004,IMX_sample00005,IMX_sample00006,IMX_sample00007,IMX_sample00008,IMX_sample00009,...,IMX_sample00077,IMX_sample00078,IMX_sample00079,IMX_sample00080,IMX_sample00081,IMX_sample00082,IMX_sample00083,IMX_sample00084,IMX_sample00085,IMX_sample00086
0,1,12,1,10,12,4,12,7,14,19,...,2,0,2,3,1,4,10,0,1,4
1,2,2670,2432,781,2381,2622,3273,3969,2597,1692,...,351,1327,1458,557,591,824,643,1403,393,1009
2,9,77,129,87,98,81,86,124,74,68,...,59,134,90,141,75,59,82,91,48,25
3,10,0,0,0,0,0,0,9,0,2,...,1,11,1,0,0,0,0,3,0,0
4,12,0,4,0,0,0,1,0,6,5,...,5,0,1,0,0,3,0,0,0,0


### Problem here: No annotation twhat gene ids are used. from the plain number my guess goes to Entrez IDs

In [118]:
import mygene

df_csv = df_csv.rename(columns={df_csv.columns[0]: 'Entrez'})

mg = mygene.MyGeneInfo()
entrez_ids = df_csv.iloc[:, 0].tolist()
query_results = mg.querymany(entrez_ids, scopes='entrezgene', fields='ensembl.gene', species='human', returnall=True)
query_results


querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-20460...done.
Finished.
42 input query terms found no hit:
	['9220', '10638', '10896', '23285', '84953', '285464', '286223', '338809', '388289', '388813', '3924


{'out': [{'query': '1',
   '_id': '1',
   '_score': 26.197802,
   'ensembl': {'gene': 'ENSG00000121410'}},
  {'query': '2',
   '_id': '2',
   '_score': 26.19706,
   'ensembl': {'gene': 'ENSG00000175899'}},
  {'query': '9',
   '_id': '9',
   '_score': 26.992018,
   'ensembl': {'gene': 'ENSG00000171428'}},
  {'query': '10',
   '_id': '10',
   '_score': 26.989302,
   'ensembl': {'gene': 'ENSG00000156006'}},
  {'query': '12',
   '_id': '12',
   '_score': 26.989254,
   'ensembl': {'gene': 'ENSG00000196136'}},
  {'query': '13',
   '_id': '13',
   '_score': 26.990025,
   'ensembl': {'gene': 'ENSG00000114771'}},
  {'query': '14',
   '_id': '14',
   '_score': 26.989716,
   'ensembl': {'gene': 'ENSG00000127837'}},
  {'query': '15',
   '_id': '15',
   '_score': 26.985958,
   'ensembl': {'gene': 'ENSG00000129673'}},
  {'query': '16',
   '_id': '16',
   '_score': 26.194178,
   'ensembl': {'gene': 'ENSG00000090861'}},
  {'query': '18',
   '_id': '18',
   '_score': 26.197521,
   'ensembl': {'gene': '

In [119]:
query_results['out']

[{'query': '1',
  '_id': '1',
  '_score': 26.197802,
  'ensembl': {'gene': 'ENSG00000121410'}},
 {'query': '2',
  '_id': '2',
  '_score': 26.19706,
  'ensembl': {'gene': 'ENSG00000175899'}},
 {'query': '9',
  '_id': '9',
  '_score': 26.992018,
  'ensembl': {'gene': 'ENSG00000171428'}},
 {'query': '10',
  '_id': '10',
  '_score': 26.989302,
  'ensembl': {'gene': 'ENSG00000156006'}},
 {'query': '12',
  '_id': '12',
  '_score': 26.989254,
  'ensembl': {'gene': 'ENSG00000196136'}},
 {'query': '13',
  '_id': '13',
  '_score': 26.990025,
  'ensembl': {'gene': 'ENSG00000114771'}},
 {'query': '14',
  '_id': '14',
  '_score': 26.989716,
  'ensembl': {'gene': 'ENSG00000127837'}},
 {'query': '15',
  '_id': '15',
  '_score': 26.985958,
  'ensembl': {'gene': 'ENSG00000129673'}},
 {'query': '16',
  '_id': '16',
  '_score': 26.194178,
  'ensembl': {'gene': 'ENSG00000090861'}},
 {'query': '18',
  '_id': '18',
  '_score': 26.197521,
  'ensembl': {'gene': 'ENSG00000183044'}},
 {'query': '19',
  '_id': '

In [120]:
query_results['out'][90]['ensembl']['gene']

'ENSG00000156110'

In [121]:
# no hit for: ['9220', '10638', '10896', '23285', '84953', '285464', '286223', '338809', '388289', '388813', '3924
mapping = {str(res['query']): res['ensembl'][0]['gene'] if isinstance(res.get('ensembl'), list) else res['ensembl']['gene'] 
           for res in query_results['out'] if 'ensembl' in res}
mapping_df = pd.DataFrame(list(mapping.items()), columns=['Entrez_Gene_ID', 'Ensembl_Gene_ID'])
mapping_df.head()

,Entrez_Gene_ID,Ensembl_Gene_ID
0,1,ENSG00000121410
1,2,ENSG00000175899
2,9,ENSG00000171428
3,10,ENSG00000156006
4,12,ENSG00000196136


Quantitation: Mapped reads were quantified as per Ensembl transcript ID as defined in
GENCODE v32 annotation. Reads were summed across Ensembl transcript IDs
mapping to Entrez gene IDs in order to compare them with other viral data assayed by
microarrays (AnnotationDbi from Bioconductor)(Pagès et al., 2017).

As entrez ID´s could match to the indexes, and are mentioned in the methods part oif the paper i guess i need to map to them 

In [122]:
df_csv['Entrez'] = df_csv['Entrez'].astype(str)
mapping_df['Entrez_Gene_ID'] = mapping_df['Entrez_Gene_ID'].astype(str)

In [123]:
merged_df = df_csv.merge(mapping_df, how='left', left_on='Entrez', right_on='Entrez_Gene_ID')
merged_df.head()


,Entrez,IMX_sample00001,IMX_sample00002,IMX_sample00003,IMX_sample00004,IMX_sample00005,IMX_sample00006,IMX_sample00007,IMX_sample00008,IMX_sample00009,...,IMX_sample00079,IMX_sample00080,IMX_sample00081,IMX_sample00082,IMX_sample00083,IMX_sample00084,IMX_sample00085,IMX_sample00086,Entrez_Gene_ID,Ensembl_Gene_ID
0,1,12,1,10,12,4,12,7,14,19,...,2,3,1,4,10,0,1,4,1,ENSG00000121410
1,2,2670,2432,781,2381,2622,3273,3969,2597,1692,...,1458,557,591,824,643,1403,393,1009,2,ENSG00000175899
2,9,77,129,87,98,81,86,124,74,68,...,90,141,75,59,82,91,48,25,9,ENSG00000171428
3,10,0,0,0,0,0,0,9,0,2,...,1,0,0,0,0,3,0,0,10,ENSG00000156006
4,12,0,4,0,0,0,1,0,6,5,...,1,0,0,3,0,0,0,0,12,ENSG00000196136


In [124]:
## As I decided to use the GOE data insteal of the .rds file. I now need metadata and only seem to see age and sex info in the regarding study accession. Therefore i try to automate the process by using Entrez from Biopython 

import pandas as pd
from Bio import Entrez



Entrez.email = "elias.schreiner@uni-tuebingen.de"
data = {
    'StudyAccession': ['GSE152641'],
    'SubjectAccession': ['GSM4622627']
}
df = pd.DataFrame(data)
df.head()


,StudyAccession,SubjectAccession
0,GSE152641,GSM4622627


In [135]:
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
from geo_utils import read_prefixes_from_metadata_txt 

metadata_txt_path = '/home/eliasschreiner/PROJEKT/2021_Thair_COVID/GSE152641_series_matrix.txt'
sample_metadata_df = read_prefixes_from_metadata_txt(metadata_txt_path)
sample_metadata_df = sample_metadata_df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)

sample_metadata_df.head()


/tmp/ipykernel_247117/1628942765.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sample_metadata_df = sample_metadata_df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)


,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,...,contact_state,contact_zip/postal_code,contact_country,data_row_count,instrument_model,library_selection,library_source,library_strategy,relation,supplementary_file_1
0,IMX_sample00001,GSM4622627,Public on Dec 10 2020,Jun 16 2020,Apr 08 2021,SRA,1,Healthy control_whole blood,Homo sapiens,Sex: Male,...,CA,94010,USA,0,Illumina NovaSeq 6000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE
1,IMX_sample00002,GSM4622628,Public on Dec 10 2020,Jun 16 2020,Apr 08 2021,SRA,1,Healthy control_whole blood,Homo sapiens,Sex: Male,...,CA,94010,USA,0,Illumina NovaSeq 6000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE
2,IMX_sample00003,GSM4622629,Public on Dec 10 2020,Jun 16 2020,Apr 08 2021,SRA,1,Healthy control_whole blood,Homo sapiens,Sex: Male,...,CA,94010,USA,0,Illumina NovaSeq 6000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE
3,IMX_sample00004,GSM4622630,Public on Dec 10 2020,Jun 16 2020,Apr 08 2021,SRA,1,Healthy control_whole blood,Homo sapiens,Sex: Male,...,CA,94010,USA,0,Illumina NovaSeq 6000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE
4,IMX_sample00005,GSM4622631,Public on Dec 10 2020,Jun 16 2020,Apr 08 2021,SRA,1,Healthy control_whole blood,Homo sapiens,Sex: Male,...,CA,94010,USA,0,Illumina NovaSeq 6000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE


In [136]:
merged_df_T = merged_df.T
merged_df_T.columns = merged_df_T.iloc[-1]
merged_df_T.rename(columns={merged_df_T.columns[0]: 'Subject_ID'}, inplace=True)
merged_df_T.reset_index(inplace=True)
merged_df_T.head()

Ensembl_Gene_ID,index,Subject_ID,ENSG00000175899,ENSG00000171428,ENSG00000156006,ENSG00000196136,ENSG00000114771,ENSG00000127837,ENSG00000129673,ENSG00000090861,...,ENSG00000210151,ENSG00000286140,ENSG00000286920,ENSG00000284791,ENSG00000242288,ENSG00000267281,ENSG00000285480,ENSG00000285292,ENSG00000286264,ENSG00000287363
0,Entrez,1,2,9,10,12,13,14,15,16,...,113219467,113455421,113523636,113523638,113939925,114108587,114483833,114483834,114841035,114841037
1,IMX_sample00001,12,2670,77,0,0,0,507,31,600,...,0,12,1004,0,0,5,0,0,0,0
2,IMX_sample00002,1,2432,129,0,4,0,598,51,592,...,0,14,1063,0,0,9,1,0,0,0
3,IMX_sample00003,10,781,87,0,0,6,807,68,770,...,0,50,993,1,0,8,0,0,0,0
4,IMX_sample00004,12,2381,98,0,0,4,607,15,1142,...,0,26,904,0,0,0,2,0,0,0


In [137]:
combi_df = merged_df_T.merge(sample_metadata_df, how='left', left_on='index', right_on='title')
combi_df.head()

,index,Subject_ID,ENSG00000175899,ENSG00000171428,ENSG00000156006,ENSG00000196136,ENSG00000114771,ENSG00000127837,ENSG00000129673,ENSG00000090861,...,contact_state,contact_zip/postal_code,contact_country,data_row_count,instrument_model,library_selection,library_source,library_strategy,relation,supplementary_file_1
0,Entrez,1,2,9,10,12,13,14,15,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IMX_sample00001,12,2670,77,0,0,0,507,31,600,...,CA,94010,USA,0,Illumina NovaSeq 6000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE
2,IMX_sample00002,1,2432,129,0,4,0,598,51,592,...,CA,94010,USA,0,Illumina NovaSeq 6000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE
3,IMX_sample00003,10,781,87,0,0,6,807,68,770,...,CA,94010,USA,0,Illumina NovaSeq 6000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE
4,IMX_sample00004,12,2381,98,0,0,4,607,15,1142,...,CA,94010,USA,0,Illumina NovaSeq 6000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE


In [141]:
combi_df.to_csv('thair_combi_covid.csv')

### R Section to compare